<a href="https://colab.research.google.com/github/natalyq1/Machine_learning1/blob/master/Optimizando_modelo_X_hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos de predicción

In [6]:
import pandas as pd
datos = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine_Learning/datos_carros.csv')
datos.head()

,precio,vendido,edad_del_modelo,Km_por_ano
0,30941.02,1,23,35085.22134
1,40557.96,1,25,12622.05362
2,89627.50,0,17,11440.79806
3,95276.14,0,8,43167.32682
4,117384.68,1,9,12770.11290


In [8]:
#datos ordenados pero de la peor manera
datos_ord = datos.sort_values('vendido', ascending=True)# primero los no vendidos
x_ord = datos_ord[['precio', 'edad_del_modelo', 'Km_por_ano']]
y_ord = datos_ord['vendido']
datos_ord.head()

,precio,vendido,edad_del_modelo,Km_por_ano
4999,74023.29,0,17,24812.80412
5322,84843.49,0,18,23095.63834
5319,83100.27,0,24,36240.72746
5316,87932.13,0,21,32249.56426
5315,77937.01,0,20,28414.50704


In [10]:
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyClassifier # modelo por defecto, una base, es muy estadistico
import numpy as np

SEED = 301
np.random.seed(SEED)

modelo = DummyClassifier()
results = cross_validate(modelo, x_ord, y_ord, cv = 10, return_train_score = False)
media= results['test_score'].mean()
desviacion_estandar = results['test_score'].std()
print("Accuracy con dummy stratified, 10 = [%.2f, %.2f]" % ((media - 2 * desviacion_estandar)*100, (media + 2 * desviacion_estandar) * 100))

Accuracy con dummy stratified, 10 = [58.00, 58.00]


In [11]:
from sklearn.tree import DecisionTreeClassifier
#en el arbol de desiciones podemos estudiar mejor las variables
SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2) # 2 posibles tomas de desicion
results = cross_validate(modelo, x_ord, y_ord, cv = 10, return_train_score=False)
media = results['test_score'].mean()
desviacion_estandar = results['test_score'].std()
print("Accuracy con cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desviacion_estandar)*100, (media + 2 * desviacion_estandar) * 100))

Accuracy con cross validation, 10 = [73.83, 77.73]


In [12]:
#generando datos aleatorios del modelo de carro
np.random.seed(SEED)
datos['modelo'] = datos.edad_del_modelo + np.random.randint(-2, 3, size=10000)
datos.modelo = datos.modelo + abs(datos.modelo.min()) + 1
datos.head()

,precio,vendido,edad_del_modelo,Km_por_ano,modelo
0,30941.02,1,23,35085.22134,26
1,40557.96,1,25,12622.05362,32
2,89627.50,0,17,11440.79806,22
3,95276.14,0,8,43167.32682,14
4,117384.68,1,9,12770.11290,13


In [13]:
#funcion
def imprime_resultados(results):
  media = results['test_score'].mean() * 100
  desviacion = results['test_score'].std() * 100
  print("Accuracy media %.2f" % media)
  print("Intervalo [%.2f, %.2f]" % (media - 2 * desviacion, media + 2 * desviacion))

In [14]:
# GroupKFold para analizar como el modelo se comporta con nuevos grupos
#validacion cruzada

from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_ord, y_ord, cv = cv, groups = datos.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy media 75.78
Intervalo [73.67, 77.90]


In [15]:
# GroupKFold en un pipeline con StandardScaler y SVC

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacion',scaler), ('estimador',modelo)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, x_ord, y_ord, cv = cv, groups = datos.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy media 76.68
Intervalo [74.28, 79.08]
